# Particle physics results

## Setup

In [1]:
import sys
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KernelDensity
from itertools import product


## Load results

In [2]:
n_runs = 5
n_chains = 4
n_trueparams = 3
remove_all_results_with_nans = False

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []
algo_dividers = []
algo_2ds = []

def add_algo(filename, add, label, lhc2d=False):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    algo_2ds.append(lhc2d)
    
    
def add_divider():
    algo_dividers.append(len(algo_filenames))
    

add_algo("flow", "_april", "2D summaries", True)
add_algo("flow", "_april", "AF")
add_algo("pie", "_april", "PIE")
add_algo("mf", "_alternate_april", "MFMF--A")
add_algo("gamf", "_april", "MFMF--OT")
add_algo("gamf", "_alternate_april", "MFMF--OTA")
add_algo("emf", "_alternate_april", "MFMF--A")

n_algos = len(algo_filenames)


In [4]:
def load(name, shape, numpyfy=True, chains=1, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add, algo_2d in zip(algo_filenames, algo_additionals, algo_2ds):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)

            try:
                this_result = np.load(
                    "{}/{}_{}_lhc{}{}{}_{}.npy".format(
                        result_dir, algo_filename, "2" if algo_2d else "14",
                        "2d" if algo_2d else "", algo_add, run_str, name
                    )
                )
                if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                    algo_results.append(this_result.reshape(shape))
                else:
                    algo_results.append(np.nan*np.ones(shape))

            except FileNotFoundError as e:
                # print(e)
                if shape is None:
                    algo_results.append(None)
                else:
                    algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    if numpyfy:
        all_results = np.array(all_results, dtype=np.float)
        
    return all_results


model_gen_x = load("samples", None, numpyfy=False)
model_gen_closure = load("samples_manifold_distance", (10000,))


In [5]:
def load_mcmc(name, shape, numpyfy=True, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add, algo_2d in zip(algo_filenames, algo_additionals, algo_2ds):
        algo_results = []
            
        for run in range(n_runs):
            run_str = "" if run == 0 else "_run{}".format(run)
            
            for trueparam in range(n_trueparams):
                trueparam_str = "" if trueparam == 0 else "_trueparam{}".format(trueparam)
            
                for chain in range(n_chains):
                    chain_str = "" if chain == 0 else "_chain{}".format(chain)

                    try:
                        this_result = np.load(
                            "{}/{}_{}_lhc{}{}{}_{}{}{}.npy".format(
                                result_dir, algo_filename, "2" if algo_2d else "14",
                                "2d" if algo_2d else "", algo_add, run_str, name, trueparam_str, chain_str
                            )
                        )
                        if (not numpyfy) or (shape is None) or np.product(this_result.shape) == np.product(shape):
                            algo_results.append(this_result.reshape(shape))
                        else:
                            algo_results.append(np.nan*np.ones(shape))

                    except FileNotFoundError as e:
                        # print(e)
                        if shape is None:
                            algo_results.append(None)
                        else:
                            algo_results.append(np.nan*np.ones(shape))
            
        all_results.append(algo_results)
    
    all_results = np.array(all_results, dtype=np.float)
    all_results = all_results.reshape([all_results.shape[0], n_runs, n_trueparams, n_chains] + list(shape))
        
    return all_results


model_posterior_samples = load_mcmc("posterior_samples", (500, 2,))
model_posterior_samples.shape  # (algo, run, true param id, chain, sample, theta component)


(7, 5, 3, 4, 500, 2)

In [6]:
def remove_nans(*results):
    passes = all([np.all(np.isfinite(result)) for result in results])
    
    if passes:
        return results
    else:
        return [np.nan * np.ones_like(result) for result in results]


def remove_nans_from_lists(*raws):
    # raws[quantity][algo]
    n_quantities = len(raws)
    n_algos = len(raws[0])
    
    for raw in raws:
        assert len(raw) == n_algos
    
    cleans = [[[] for _ in range(n_algos)] for _ in range(n_quantities)]
    
    for i in range(n_algos):
        for k in range(n_runs):
            clean = remove_nans(*[raw[i][k] for raw in raws])
            for j in range(n_quantities):
                cleans[j][i].append(clean[j])
            
    cleans = [np.array(clean) for clean in cleans]
    
    # cleans[quantity][algo]
    return cleans
     
    
if remove_all_results_with_nans:
    model_gen_closure, model_posterior_samples = remove_nans_from_lists(model_gen_closure, model_posterior_samples)


## Calculate metrics

In [7]:
min_logp = -100.
model_gen_mean_closure = np.mean(model_gen_closure, axis=2)


In [8]:
bandwidth = 0.25
true_param_points = np.array([[0.,0.], [0.5, 0.], [-1., -1.]])

model_true_log_posteriors = []

for algo, run, trueparam in product(range(n_algos), range(n_runs), range(n_trueparams)):
    mcmcs = model_posterior_samples[algo, run, trueparam].reshape((-1, 2))
    mcmcs = mcmcs[np.all(np.isfinite(mcmcs), axis=-1)]
    
    if len(mcmcs) == 0:
        model_true_log_posteriors.append(np.nan)
        continue
        
    print(algo, run, trueparam)
    kde = KernelDensity(kernel="gaussian", bandwidth=bandwidth)
    kde.fit(mcmcs)
    model_true_log_posteriors.append(kde.score(true_param_points[trueparam].reshape((1, 2))))

model_true_log_posteriors = np.array(model_true_log_posteriors).reshape((n_algos, n_runs, n_trueparams)) 


0 0 0
0 0 1
0 0 2
0 1 0
0 1 1
0 1 2
0 2 0
0 2 1
0 2 2
0 3 0
0 3 1
0 3 2
0 4 0
0 4 1
0 4 2
1 0 0
1 0 1
1 0 2
1 1 0
1 1 1
1 1 2
1 2 0
1 2 1
1 2 2
1 3 0
1 3 1
1 3 2
1 4 0
1 4 1
1 4 2
2 0 0
2 0 1
2 0 2
2 1 0
2 1 1
2 1 2
2 2 0
2 2 1
2 2 2
3 0 0
3 0 1
3 0 2
3 1 0
3 1 1
3 1 2
3 2 0
3 2 1
3 2 2
6 0 0
6 0 1
6 0 2
6 2 0
6 2 1
6 2 2


## Best metrics

In [9]:
model_true_log_posteriors[:,0,:]

array([[ 0.90962074,  0.44834521, -1.53297605],
       [-2.06671826, -2.10419179, -6.03832848],
       [ 0.84044445, -1.92213525, -1.7525699 ],
       [-1.55394298, -2.09669399, -1.59071453],
       [        nan,         nan,         nan],
       [        nan,         nan,         nan],
       [-1.73578427, -1.3277774 , -2.69542223]])

In [10]:
np.nanmean(model_true_log_posteriors.reshape((n_algos, -1)), axis=-1)

/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


array([-0.1452924 , -2.44951515, -1.70756766, -2.13749922,         nan,
               nan, -1.88817743])

In [11]:
best_closure, best_posterior = -1, -1

best_closure = np.nanargmin(np.nanmean(model_gen_mean_closure, axis=1))
print(algo_labels[best_closure])

best_posterior = np.nanargmax(np.nanmean(model_true_log_posteriors.reshape((n_algos, -1)), axis=-1))
print(algo_labels[best_posterior])


AF
2D summaries


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


## Print metrics

In [12]:
def print_results(
    include_err=False, include_n_runs=False, include_runs=False,
    l_label=max([len(l) for l in algo_labels]), l_mean=5, l_err=2,
    median=False, latex=False, after_decs=(3,2)
):
    # How to format the numbers
    l_result = (
        l_mean + int(include_err) * (2 + l_err)
        + int(include_n_runs) * 4
        + int(include_runs) * (3 + n_runs*l_mean + (n_runs - 1)*2)
    )
    l_total = l_label + 1 + 2 * (3 + l_result)
        
    # Divider
    empty_result = "" if latex else " "*(l_result + 1)
    col_divider = "&" if latex else "|"
    line_end = r"\\" if latex else ""
    block_divider = r"\midrule" if latex else "-"*l_total
    
    def _f(val, after_dec, best=False):
        if not np.any(np.isfinite(val)):
            return empty_result
        
        if median:
            result = "{:>{}.{}f}".format(np.nanmedian(val), l_mean, after_dec)
        else:
            result = "{:>{}.{}f}".format(np.nanmean(val), l_mean, after_dec)
            
        if latex and best:
            result = r"\textbf{" + result + "}"
            
        if include_err:
            err_str = "({:0>{}d})".format(int(10**after_dec * np.nanstd(val) / np.sum(np.isfinite(val))**0.5), l_err)
            if latex:
                result += r"\,\textcolor{dark-gray}{" + err_str + "}"
            else:
                result += err_str
            
        if include_n_runs:
            result += " [{:1n}]".format(np.sum(np.isfinite(val)))
        if include_runs:
            result += " [{:>{}.{}f}, ".format(np.nanmean(val[0]), l_mean, after_dec)
            result += "{:>{}.{}f}, ".format(np.nanmean(val[1]), l_mean, after_dec)
            result += "{:>{}.{}f}, ".format(np.nanmean(val[2]), l_mean, after_dec)
            result += "{:>{}.{}f}, ".format(np.nanmean(val[3]), l_mean, after_dec)
            result += "{:>{}.{}f}]".format(np.nanmean(val[4]), l_mean, after_dec)
            
        if (not latex) and best:
            result += "*"
        elif (not latex) and (not best):
            result += " "
            
        return result
    
    # Header
    print(
        "{2:<{0}.{0}s} {5} {3:>{1}.{1}s} {5} {4:>{1}.{1}s} {6}".format(
            l_label, l_result, "", "Closure", "Posterior mean error", col_divider, line_end
        )
    )
    print(block_divider)

    # Iterate over methods
    for i, (label, closure, posterior) in enumerate(zip(
        algo_labels, model_gen_mean_closure, model_true_log_posteriors
    )):
        # Divider
        if i in algo_dividers and show_all:
            print(block_divider)
            
        # Print results
        print("{1:<{0}.{0}s} {4} {2}{4} {3} {5}".format(
            l_label, label,
            _f(closure, after_decs[0], i==best_closure), _f(posterior, after_decs[1], i==best_posterior),
            col_divider, line_end
        ))


In [13]:
print_results(include_runs=True)

             |                                   Closure |                      Posterior mean error 
-----------------------------------------------------------------------------------------------------
2D summaries |                                           | -0.15 [-0.06, -0.09, -0.26, -0.39,  0.07]* 
AF           | 0.018 [0.016, 0.017, 0.017, 0.017, 0.022]*| -2.45 [-3.40, -2.67, -1.79, -2.26, -2.12]  
PIE          | 0.030 [0.050, 0.025, 0.015,   nan,   nan] | -1.71 [-0.94, -2.41, -1.77,   nan,   nan]  
MFMF--A      | 0.315 [0.264, 0.319, 0.363,   nan,   nan] | -2.14 [-1.75, -2.12, -2.55,   nan,   nan]  
MFMF--OT     |                                           |                                            
MFMF--OTA    |                                           |                                            
MFMF--A      | 0.423 [0.472, 0.362, 0.436,   nan,   nan] | -1.89 [-1.92,   nan, -1.86,   nan,   nan]  


/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: Mean of empty slice
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: Mean of empty slice
/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice


In [14]:
print_results(include_err=True)

             |   Closure | Posterior 
-------------------------------------
2D summaries |           | -0.15(33)* 
AF           | 0.018(01)*| -2.45(38)  
PIE          | 0.030(08) | -1.71(94)  
MFMF--A      | 0.315(23) | -2.14(17)  
MFMF--OT     |           |            
MFMF--OTA    |           |            
MFMF--A      | 0.423(26) | -1.89(22)  


In [15]:
print_results(include_err=True, latex=True)

             &   Closure & Posterior \\
\midrule
2D summaries & & \textbf{-0.15}\,\textcolor{dark-gray}{(33)} \\
AF           & \textbf{0.018}\,\textcolor{dark-gray}{(01)}& -2.45\,\textcolor{dark-gray}{(38)} \\
PIE          & 0.030\,\textcolor{dark-gray}{(08)}& -1.71\,\textcolor{dark-gray}{(94)} \\
MFMF--A      & 0.315\,\textcolor{dark-gray}{(23)}& -2.14\,\textcolor{dark-gray}{(17)} \\
MFMF--OT     & &  \\
MFMF--OTA    & &  \\
MFMF--A      & 0.423\,\textcolor{dark-gray}{(26)}& -1.89\,\textcolor{dark-gray}{(22)} \\
